In [ ]:
!pip install git+ssh://git@bitbucket.org/dreamplug-backend/test-custom-inferencing@0.0.5

In [ ]:
%%writefile hooks.py
from kuberinference import pre_processing, post_processing

def input_fn(X):
    return pre_processing.pre_process(X)

def output_fn(Y):
    return post_processing.post_process(Y)

In [ ]:
############################################## CUSTOM INFERENCING CODE #############################################
from kuberinference import inferencing
import mlflow

class ClusterKuberInference(mlflow.pyfunc.PythonModel):

    def __init__(self):
        self.model = inferencing.KuberInference()
    
    def predict(self, X):
        return self.model.predict(X)

In [ ]:
######## TEST INFERENCE ###################
mlflow.set_tracking_uri("http://model-repository.stg.dreamplug.net/")

from kuberinference.pre_processing import pre_process
from kuberinference.post_processing import post_process

X = {
    "user_id": "bff2781f-123f-412c-9341-7055c18437ba",
    "instru_id": ["NB_HDFC","NB_HDFC1"],
    "amount": 500,
    "lob": "6f6199c0-3005-4978-ac75-fbe591e56760"
    }
X = pre_process(X)
Y = ClusterKuberInference().predict(X)
Y = post_process(Y)
print(Y)

In [ ]:
model = ClusterKuberInference()
############################## LOG INFERENCE MODEL ###########################
mlflow.set_tracking_uri("http://model-repository.stg.dreamplug.net/")
mlflow.set_experiment("kuber-inference-base-model")
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(python_model=model, artifact_path="model", 
                             registered_model_name="KUBER-INFERENCE-MODEL", 
                            extra_pip_requirements=["git+ssh://git@bitbucket.org/dreamplug-backend/test-custom-inferencing@0.0.5"]
                            )
    mlflow.log_artifacts("hooks.py", "model")